# Survey Vlaamse Meesters

## Data Import

In [1]:
import pandas as pd
import numpy as np

file="data/Onbewerkte-gegevens-24513-51204-201741010819-TH.csv"
raw_df = pd.read_csv(file, encoding = "ISO-8859-1", sep=";")
df = raw_df.copy()

## Cleaning and first glance

In [2]:
# remove strange characters from column headers
columns = list(df)
columns[40] = "8.1. Reisprogramma's op tv/radio"
columns[48] = "8.9. Officiële website van toeristische organisatie (bv. VisitBritain, VisittheUSA,...)"
columns[81] = "12.5. Interactief aanbod (virtual reality, doe-activiteiten, gebruik van touchscreens,...)"
columns[90] = "14.6. Geboortehuis, woning,... van een kunstenaar"
columns[99] = "16. Hieronder ziet u een aantal schilderijen. Kent u dit werk?"
columns[101] = "18. Kent u dit werk?"
columns[103] = "20. Kent u dit werk?"
columns[105] = "22. Kent u dit werk?"
columns[139] = "26.2. Inspiratieplekken (landschap, leefomgeving,...) van kunstenaars bezoeken/verkennen/ontdekken is interessant."
columns[142] = "26.5. Een beleving (themawandeling, doe-activiteit,...) over een kunstenaar spreekt me aan."
columns[143] = "27. Kent u de term 'Vlaamse Meesters'?"

df.columns = columns
list(df)

['Respondent',
 'Taal',
 'Datum geantwoord',
 'Distributiemethode',
 'Einde bereikt',
 'Invultijd (seconden)',
 'Browser',
 'Besturingssysteem',
 'IP',
 'Locatie (via IP)',
 'Verwijzende sites',
 '1. ID',
 '2. Country',
 '3. Study number',
 '4. Wat is uw geboortejaar?',
 '5. Bent u in de afgelopen 3 jaar wel eens voor vakantie in het buitenland geweest?',
 '6.1. Strandvakantie',
 '6.2. Stedentrip',
 '6.3. Wintersportvakantie',
 '6.4. Wandelvakantie',
 '6.5. Fietsvakantie',
 '6.6. Wellnessvakantie',
 '6.7. Gastronomische of culinaire vakantie',
 '6.8. Vakantie in het groen met (af en toe) uitstappen',
 '6.9. Rondreis',
 '6.10. Cruise',
 '6.11. Culturele reis',
 '6.12. Ander, gelieve te specificeren',
 '6.13. Geen van bovenstaande',
 '7.1. Afstand',
 '7.2. Historisch erfgoed',
 '7.3. Eten',
 '7.4. Cultuur',
 '7.5. Natuur',
 '7.6. Prijs',
 '7.7. Kust',
 '7.8. Sport',
 '7.9. Festival, evenement of tentoonstelling',
 '7.10. Kunst',
 '7.11. Variatie aan activiteiten',
 "8.1. Reisprogramma's 

In [3]:
df["Datum geantwoord"] = pd.to_datetime(df.loc[:,"Datum geantwoord"], infer_datetime_format=True)
df["Distributiemethode"]=df["Distributiemethode"].astype('category')
df["Browser"]=df["Browser"].astype('category')
df["Besturingssysteem"]=df["Besturingssysteem"].astype('category')

df.iloc[:3, 0:8]

,Respondent,Taal,Datum geantwoord,Distributiemethode,Einde bereikt,Invultijd (seconden),Browser,Besturingssysteem
0,22,nl,2015-12-16 10:17:00,Via panel provider,1,312,Google Chrome,Windows 10
1,23,nl,2015-12-16 10:29:00,Via panel provider,1,919,Firefox,Windows Vista
2,24,nl,2015-12-16 10:44:00,Via panel provider,1,154,Google Chrome,Windows 7


In [4]:
df.iloc[:3, 8:15]

,IP,Locatie (via IP),Verwijzende sites,1. ID,2. Country,3. Study number,4. Wat is uw geboortejaar?
0,84.31.26.69,NaN,http://www.globaltestmarket.com/survey/onsurve...,6da3a071da328d79,Netherlands,394334,1968
1,77.173.234.228,"Putte, Nederland",http://www.globaltestmarket.com/survey/onsurve...,6456a4ddd07e1021,Netherlands,394334,1981
2,89.146.11.36,"Westervoort, Nederland",http://www.globaltestmarket.com/survey/onsurve...,02e8569ecb3b5276,Netherlands,394334,1991


In [5]:
# change to boolean values

df["5. Bent u in de afgelopen 3 jaar wel eens voor vakantie in het buitenland geweest?"] = raw_df["5. Bent u in de afgelopen 3 jaar wel eens voor vakantie in het buitenland geweest?"] == 'Ja'
df["6.1. Strandvakantie"] = raw_df["6.1. Strandvakantie"] == 'Strandvakantie'
df["6.2. Stedentrip"] = raw_df["6.2. Stedentrip"] == 'Stedentrip'
df["6.3. Wintersportvakantie"] = raw_df["6.3. Wintersportvakantie"] == 'Wintersportvakantie'
df["6.4. Wandelvakantie"] = raw_df["6.4. Wandelvakantie"] == 'Wandelvakantie'
df["6.5. Fietsvakantie"] = raw_df["6.5. Fietsvakantie"] == 'Fietsvakantie'
df["6.6. Wellnessvakantie"] = raw_df["6.6. Wellnessvakantie"] == 'Wellnessvakantie'
df["6.7. Gastronomische of culinaire vakantie"] = raw_df["6.7. Gastronomische of culinaire vakantie"] == 'Gastronomische of culinaire vakantie'
df["6.8. Vakantie in het groen met (af en toe) uitstappen"] = raw_df["6.8. Vakantie in het groen met (af en toe) uitstappen"] == 'Vakantie in het groen met (af en toe) uitstappen'
df["6.9. Rondreis"] = raw_df["6.9. Rondreis"] == 'Rondreis'
df["6.10. Cruise"] = raw_df["6.10. Cruise"] == 'Cruise'
df["6.11. Culturele reis"] = raw_df["6.11. Culturele reis"] == 'Culturele reis'
df["6.13. Geen van bovenstaande"] = raw_df["6.13. Geen van bovenstaande"] == 'Geen van bovenstaande'
df.iloc[:3, 15:29]


,5. Bent u in de afgelopen 3 jaar wel eens voor vakantie in het buitenland geweest?,6.1. Strandvakantie,6.2. Stedentrip,6.3. Wintersportvakantie,6.4. Wandelvakantie,6.5. Fietsvakantie,6.6. Wellnessvakantie,6.7. Gastronomische of culinaire vakantie,6.8. Vakantie in het groen met (af en toe) uitstappen,6.9. Rondreis,6.10. Cruise,6.11. Culturele reis,"6.12. Ander, gelieve te specificeren",6.13. Geen van bovenstaande
0,True,False,False,False,False,False,False,False,True,False,False,False,NaN,False
1,True,True,False,False,True,False,False,False,False,True,False,False,NaN,False
2,True,True,False,False,False,False,False,False,False,False,False,False,NaN,False


In [26]:
(df.iloc[:, 15:29].sum()/df.iloc[:, 15:29].count()).sort_values(ascending = False)

5. Bent u in de afgelopen 3 jaar wel eens voor vakantie in het buitenland geweest?    0.999941
6.1. Strandvakantie                                                                   0.595824
6.2. Stedentrip                                                                       0.536882
6.11. Culturele reis                                                                  0.229529
6.9. Rondreis                                                                         0.220471
6.8. Vakantie in het groen met (af en toe) uitstappen                                 0.198529
6.6. Wellnessvakantie                                                                 0.149529
6.4. Wandelvakantie                                                                   0.137176
6.10. Cruise                                                                          0.136941
6.3. Wintersportvakantie                                                              0.112000
6.7. Gastronomische of culinaire vakantie         

In [6]:
df["6.12. Ander, gelieve te specificeren"].dropna().unique()

array(['familiebezoek', 'familie bezoek', 'Bezoek vrienden', ...,
       'Disney e Universal', '???????????', 'meeting relatives'], dtype=object)

In [7]:
df["7.1. Afstand"] = raw_df["7.1. Afstand"] == 'Afstand'
df["7.2. Historisch erfgoed"] = raw_df["7.2. Historisch erfgoed"] == 'Historisch erfgoed'
df["7.3. Eten"] = raw_df["7.3. Eten"] == 'Eten'
df["7.4. Cultuur"] = raw_df["7.4. Cultuur"] == 'Cultuur'
df["7.5. Natuur"] = raw_df["7.5. Natuur"] == 'Natuur'
df["7.6. Prijs"] = raw_df["7.6. Prijs"] == 'Prijs'
df["7.7. Kust"] = raw_df["7.7. Kust"] == 'Kust'
df["7.8. Sport"] = raw_df["7.8. Sport"] == 'Sport'
df["7.9. Festival, evenement of tentoonstelling"] = raw_df["7.9. Festival, evenement of tentoonstelling"] == 'Festival, evenement of tentoonstelling'
df["7.10. Kunst"] = raw_df["7.10. Kunst"] == 'Kunst'
df["7.11. Variatie aan activiteiten"] = raw_df["7.11. Variatie aan activiteiten"] == 'Variatie aan activiteiten'

df.iloc[:3, 29:40]

,7.1. Afstand,7.2. Historisch erfgoed,7.3. Eten,7.4. Cultuur,7.5. Natuur,7.6. Prijs,7.7. Kust,7.8. Sport,"7.9. Festival, evenement of tentoonstelling",7.10. Kunst,7.11. Variatie aan activiteiten
0,True,False,False,False,True,True,False,False,False,False,False
1,False,False,True,False,False,False,True,False,True,False,False
2,True,False,False,False,False,True,True,False,False,False,False


In [25]:
(df.iloc[:, 29:40].sum()/df.iloc[:, 29:40].count()).sort_values(ascending = False)

7.6. Prijs                                     0.568941
7.4. Cultuur                                   0.448412
7.5. Natuur                                    0.406941
7.3. Eten                                      0.396412
7.2. Historisch erfgoed                        0.340118
7.7. Kust                                      0.250529
7.1. Afstand                                   0.234882
7.11. Variatie aan activiteiten                0.189118
7.9. Festival, evenement of tentoonstelling    0.065882
7.10. Kunst                                    0.049706
7.8. Sport                                     0.049059
dtype: float64

In [37]:
df["8.1. Reisprogramma's op tv/radio"] = pd.Categorical(raw_df.iloc[:,40], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.2. Websites van (online) touroperators / reisagenten"] = pd.Categorical(raw_df["8.2. Websites van (online) touroperators / reisagenten"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.4. Websites met beoordelingen/reviews van reizigers"] = pd.Categorical(raw_df["8.4. Websites met beoordelingen/reviews van reizigers"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.5. Reismagazines"] = pd.Categorical(raw_df["8.5. Reismagazines"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.6. Vrienden, familie en kennissen"] = pd.Categorical(raw_df["8.6. Vrienden, familie en kennissen"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.7. Reisbijlages/ - artikelen in kranten"] = pd.Categorical(raw_df["8.7. Reisbijlages/ - artikelen in kranten"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.8. Cultuurmagazines"] = pd.Categorical(raw_df["8.8. Cultuurmagazines"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.9. Officiële website van toeristische organisatie (bv. VisitBritain, VisittheUSA,...)"] = pd.Categorical(raw_df.iloc[:,48], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)
df["8.10. Sociale media"] = pd.Categorical(raw_df["8.10. Sociale media"], 
            categories=['Nooit', 'Af en toe', 'Regelmatig', 'Vaak'],
            ordered=True)

df.iloc[:3, 40:50]

,8.1. Reisprogramma's op tv/radio,8.2. Websites van (online) touroperators / reisagenten,8.3. Websites waarop consumenten reiservaringen en verhalen delen/uitwisselen (reisblogs),8.4. Websites met beoordelingen/reviews van reizigers,8.5. Reismagazines,"8.6. Vrienden, familie en kennissen",8.7. Reisbijlages/ - artikelen in kranten,8.8. Cultuurmagazines,"8.9. Officiële website van toeristische organisatie (bv. VisitBritain, VisittheUSA,...)",8.10. Sociale media
0,Nooit,Nooit,Nooit,Nooit,Nooit,Af en toe,Nooit,Nooit,Nooit,Nooit
1,Regelmatig,Regelmatig,Af en toe,Af en toe,Af en toe,Regelmatig,Af en toe,Af en toe,Af en toe,Regelmatig
2,Nooit,Vaak,Vaak,Vaak,Af en toe,Regelmatig,Af en toe,Af en toe,Af en toe,Af en toe


In [16]:
df["9.1. Facebook"] = raw_df["9.1. Facebook"] == 'Facebook'
df["9.2. Twitter"] = raw_df["9.2. Twitter"] == 'Twitter'
df["9.3. Youtube"] = raw_df["9.3. Youtube"] == 'Youtube'
df["9.4. Pinterest"] = raw_df["9.4. Pinterest"] == 'Pinterest'
df["9.5. Instagram"] = raw_df["9.5. Instagram"] == 'Instagram'
df["9.6. LinkedIn"] = raw_df["9.6. LinkedIn"] == 'LinkedIn'
df["9.7. Google+"] = raw_df["9.7. Google+"] == 'Google+'
df["9.8. Flickr"] = raw_df["9.8. Flickr"] == 'Flickr'
df["9.9. Tumblr"] = raw_df["9.9. Tumblr"] == 'Tumblr'
df["9.10. Vine"] = raw_df["9.10. Vine"] == 'Vine'
df["9.11. Snapchat"] = raw_df["9.11. Snapchat"] == 'Snapchat'
df["9.12. Whatsapp"] = raw_df["9.12. Whatsapp"] == 'Whatsapp'
df["9.13. Foursquare"] = raw_df["9.13. Foursquare"] == 'Foursquare'
df["9.14. Orkut"] = raw_df["9.14. Orkut"] == 'Orkut'
df["9.15. Weibo"] = raw_df["9.15. Weibo"] == 'Weibo'
df["9.16. We Chat"] = raw_df["9.16. We Chat"] == 'We Chat'
df["9.17. VKontakte"] = raw_df["9.17. VKontakte"] == 'VKontakte'
df["9.19. Ik gebruik geen sociale media"] = raw_df["9.19. Ik gebruik geen sociale media"] == 'Ik gebruik geen sociale media'
df.iloc[:3, 50:69]

,9.1. Facebook,9.2. Twitter,9.3. Youtube,9.4. Pinterest,9.5. Instagram,9.6. LinkedIn,9.7. Google+,9.8. Flickr,9.9. Tumblr,9.10. Vine,9.11. Snapchat,9.12. Whatsapp,9.13. Foursquare,9.14. Orkut,9.15. Weibo,9.16. We Chat,9.17. VKontakte,"9.18. Ander, gelieve te specificeren",9.19. Ik gebruik geen sociale media
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,True
1,True,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,NaN,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,False


In [24]:
(df.iloc[:, 50:69].sum()/df.iloc[:, 50:69].count()).sort_values(ascending = False)

9.1. Facebook                           0.664176
9.3. Youtube                            0.367353
9.12. Whatsapp                          0.317647
9.7. Google+                            0.223118
9.2. Twitter                            0.204882
9.5. Instagram                          0.190647
9.19. Ik gebruik geen sociale media     0.134706
9.6. LinkedIn                           0.104176
9.11. Snapchat                          0.074588
9.16. We Chat                           0.071294
9.4. Pinterest                          0.065588
9.15. Weibo                             0.046235
9.17. VKontakte                         0.046000
9.9. Tumblr                             0.028882
9.8. Flickr                             0.024353
9.13. Foursquare                        0.019824
9.10. Vine                              0.019765
9.14. Orkut                             0.013118
9.18. Ander, gelieve te specificeren         NaN
dtype: float64

In [38]:
df["10.1. In eigen land"] = pd.Categorical(raw_df["10.1. In eigen land"], 
            categories=['Nooit', 'Zelden', 'Af en toe (een keer per jaar)', 'Regelmatig (meerdere keren per jaar)', 'Vaak (maandelijks)'],
            ordered=True)
df["10.2. In buitenland"] = pd.Categorical(raw_df["10.2. In buitenland"], 
            categories=['Nooit', 'Zelden', 'Af en toe (een keer per jaar)', 'Regelmatig (meerdere keren per jaar)', 'Vaak (maandelijks)'],
            ordered=True)
df.iloc[:3, 69:71]

,10.1. In eigen land,10.2. In buitenland
0,Zelden,Nooit
1,Af en toe (een keer per jaar),Af en toe (een keer per jaar)
2,Zelden,Zelden


In [ ]:
df.iloc[:3, 77:85]

In [ ]:
df.iloc[:3, 85:94]

In [ ]:
df.iloc[:3, 94:99]

In [ ]:
df.iloc[:3, 99:107]

In [ ]:
df.iloc[:3, 107:125]

In [ ]:
df.iloc[:3, 125:138]

In [ ]:
df.iloc[:3, 138:143]

In [ ]:
df.iloc[:3, 143:148]

In [ ]:
df.iloc[:3, 148:165]

In [ ]:
df.iloc[:3, 165:171]

## Data Wrangling
### dates to datetime

In [ ]:

df[:3]

## List features


In [ ]:
list(df)

## Basic visualisation of features

In [ ]:
import matplotlib.pyplot as plt
plt.matplotlib.style.use('ggplot')

### Taal

In [ ]:
plt_df = df[["Respondent","Taal"]].groupby("Taal").Respondent.nunique()
plt_df = plt_df.sort_values(ascending =False)

# Strings
xlab = 'Taal Respondent'
ylab = 'Aantal respondenten'
title = 'Distributie taal over respondenten'

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Create plot
plt_df.plot.bar();
    
# Render plot
plt.show()

## Datum geantwoord

In [ ]:
plt_df = df[["Taal", "Datum geantwoord"]].copy()
plt_df['count'] = 1;
plt_df = plt_df.sort_values(by="Datum geantwoord")
plt_df['cumcount'] = plt_df.groupby(by=["Taal"]).cumsum()

plt_df = plt_df.drop(['count'], axis =1)


In [ ]:
# build plot language per language

for lan in plt_df.Taal.unique():
    x=plt_df.loc[plt_df.Taal == lan, ['Datum geantwoord']]
    y=plt_df.loc[plt_df.Taal == lan, ['cumcount']]
    plt.plot (x, y, label=lan)

# Strings
xlab = 'Tijd'
ylab = 'Aantal respondenten'
title = 'recrutering - vooruitgang per taal'

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Add legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.gcf().autofmt_xdate()
# Render plot
plt.show()
    

# Einde bereikt

In [ ]:
print(df["Einde bereikt"].unique())

# Invultijd

In [ ]:
df["Invultijd (seconden)"].plot.hist()

# Strings
xlab = 'Invultijd (s)'
ylab = 'Frequentie'
title = 'Histogram invultijd'

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Render plot
plt.show()

In [ ]:
df_plot = (df.loc[df["Invultijd (seconden)"]> 3600, "Invultijd (seconden)"]/3600)
df_plot = df_plot.rename("")
df_plot.plot.box()

# Strings
xlab = 'Respondenten'
ylab = 'Invultijd (h)'
title = "Invultijd voor respondenten die langer dan 1 uur over de survey deden n=" + str(df_plot.size)

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Render plot
plt.show()

In [ ]:
plt_df = df.loc[df["Invultijd (seconden)"]> 3600, ["Respondent","Taal"]].groupby("Taal").Respondent.nunique()
plt_df = plt_df.sort_values(ascending =False)

# Strings
xlab = 'Taal Respondent'
ylab = 'Aantal respondenten'
title = 'Distributie taal over respondenten die meer dan 1 uur over de survey deden'

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Create plot
plt_df.plot.bar();
    
# Render plot
plt.show()
plt_df

In [ ]:
df_rel = (df.loc[df["Invultijd (seconden)"] > 3600,["Respondent","Taal"]].groupby("Taal").Respondent.count() * 100/
 df[["Respondent","Taal"]].groupby("Taal").Respondent.count())
df_rel = df_rel.sort_values(ascending =False)
df_rel

In [ ]:

# Strings
xlab = 'Taal Respondent'
ylab = '% respondenten'
title = '% respondenten die meer dan 1 uur over de survey deden per taalgroep'

# Add axis labels
plt.xlabel(xlab)
plt.ylabel(ylab)

# Add title
plt.title(title)

# Create plot
df_rel.plot.bar();
    
# Render plot
plt.show()